# Click2buy and buy2buy co-visitation matrixes
This notebook prepares click-to-buy and buy-to-buy co-visitation matrixes. It is used to generate candidates and calculate features for the carts and orders models.
There are three different outputs both for the cross-validation and for test datasets. There is full version for both matrixes and a version of click-to-buy matrix limited to top 70 records, which is used for candidate generation.


## Imports and definitions

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
  
import gc
from humanize import naturalsize

# functions and classes common for several notebooks of current project
import otto_common

In [2]:
class CalculateCovisitationMatrixClick2Buy(otto_common.CalculateCovisitationMatrix):
    '''
    Child class with logic specific to this type of covisitation matrix.
    It counts all aids that were either carted or ordered within certain time after some aid was clicked.
    The weight value is calculated in a way that makes pairs of events with smaller time difference more important.
    '''

    def calculate_weights(self, df_i):
        df_i = df_i.sort_values(['session','ts'],ascending=[True,False])
        df_i = df_i.reset_index(drop=True)
        df_i = df_i.merge(df_i,on='session')
        df_i = df_i.loc[ (df_i.ts_y >= df_i.ts_x) & (df_i.type_y > 0)]
        df_i['time_diff'] = df_i.ts_y - df_i.ts_x
        df_i = df_i.loc[ (df_i.time_diff).abs() < 10 * 60 * 60]
        df_i = df_i.groupby(['session','aid_x', 'aid_y']).agg({'time_diff':'min'}).reset_index()
        df_i['wgt'] = (60 * 60)/(60 * 60 + df_i.time_diff)
        df_i = df_i[['aid_x','aid_y','wgt']]
        df_i = df_i.loc[df_i['aid_x'].isin(self.aid_list)]
        df_i.wgt = df_i.wgt.astype('float32')
        df_i = self.groupby_reset_and_reduce(df_i)
        return df_i

In [3]:
class CalculateCovisitationMatrixBuy2Buy(otto_common.CalculateCovisitationMatrix):
    '''
    Child class with logic specific to this type of covisitation matrix.
    It counts all aids that were either carted or ordered within certain time after some aid was either carted or ordered.
    The weight value is the same for each pair of events.
    '''

    def calculate_weights(self, df_i):
        df_i = df_i.loc[df_i['type'] > 0]
        df_i = df_i.sort_values(['session','ts'],ascending=[True,False])
        df_i = df_i.reset_index(drop=True)
        df_i = df_i.merge(df_i,on='session')
        df_i = df_i.loc[ ((df_i.ts_x - df_i.ts_y).abs()< 5 * 24 * 60 * 60)]
        df_i = df_i[['session', 'aid_x', 'aid_y']].drop_duplicates(['session', 'aid_x', 'aid_y']) 
        df_i['wgt'] = 1
        df_i = df_i[['aid_x','aid_y','wgt']]
        df_i = df_i.loc[df_i['aid_x'].isin(self.aid_list)]
        df_i.wgt = df_i.wgt.astype('float32')
        df_i = self.groupby_reset_and_reduce(df_i)
        return df_i

## Co-visitation matrices for cross-validation dataset

In [4]:
# Make a list of AIDs, that appear in cross-validation sets.
# aid_x is filtered to only include aids from the list, while aid_y is not filtered
# This reduces size of resulting matrix.
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/cv_inputs.parquet'
trunked_sessions_path2 = '/kaggle/input/otto-prepare-cv/cv_inputs2.parquet'
cv_list = otto_common.build_aid_list(trunked_sessions_path, trunked_sessions_path2)

In [5]:
# Input parameters and click2buy matrix calculation for cross-validation dataset.
click_data_cv = '/kaggle/input/otto-prepare-cv/cv_train.parquet'
n_splits = 200
n_splits_concat = 30
top_n = 70

covisitation_click2buy_cv = CalculateCovisitationMatrixClick2Buy(n_splits, n_splits_concat, cv_list)
df_matrix_cv = covisitation_click2buy_cv.generate_covisitation_matrix(click_data_cv)
df_matrix_cv.to_parquet('matrix_click2buy_cv.parquet')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [6]:
# Reduce click2buy matrix to top 70, check dataframe size and export to file.
df_matrix_cv = covisitation_click2buy_cv.get_top_n(df_matrix_cv, top_n)
size = df_matrix_cv.memory_usage(deep='True').sum()
print(naturalsize(size))
df_matrix_cv.to_parquet('matrix_click2buy_cv_top_n.parquet')

571.5 MB


In [7]:
del df_matrix_cv, covisitation_click2buy_cv
gc.collect()

21

In [8]:
# Input parameters and buy2buy matrix calculation for cross-validation dataset.
# As about 90% of all events in history are clicks, so it is possible to use much lower values for number of splits and speed up the calculation of 
# buy2buy matrix.

n_splits = 50
n_splits_concat = 5

covisitation_buy2buy_cv = CalculateCovisitationMatrixBuy2Buy(n_splits, n_splits_concat, cv_list)
df_matrix_b2b_cv = covisitation_buy2buy_cv.generate_covisitation_matrix(click_data_cv)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [9]:
# Check dataframe size and export to file
size = df_matrix_b2b_cv.memory_usage(deep='True').sum()
print(naturalsize(size))
df_matrix_b2b_cv.to_parquet('matrix_buy2buy_cv.parquet')

del df_matrix_b2b_cv, cv_list, covisitation_buy2buy_cv
gc.collect()

1.2 GB


21

## Co-visitation matrices for test dataset

In [10]:
# Make a list of AIDs, that appear in test dataset.
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/test.parquet'
full_aid_list = otto_common.build_aid_list(trunked_sessions_path)

In [11]:
# Input parameters and click2buy matrix calculation for test dataset.
click_data_full = '/kaggle/input/otto-prepare-cv/train_full.parquet'
n_splits = 400
n_splits_concat = 40
top_n = 70

covisitation_click2buy_full = CalculateCovisitationMatrixClick2Buy(n_splits, n_splits_concat, full_aid_list)
df_matrix_full = covisitation_click2buy_full.generate_covisitation_matrix(click_data_full)
df_matrix_full.to_parquet('matrix_click2buy_test.parquet')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
# Reduce click2buy matrix to top 70, check dataframe size and export to file.
df_matrix_full.to_parquet('matrix_click2buy_test.parquet')
size = df_matrix_full.memory_usage(deep='True').sum()
print(naturalsize(size))
df_matrix_full.to_parquet('matrix_click2buy_test_top_n.parquet')

del df_matrix_full, covisitation_click2buy_full
gc.collect()

2.0 GB


21

In [13]:
# Input parameters and buy2buy matrix calculation for test dataset.
n_splits = 70
n_splits_concat = 10

covisitation_buy2buy_full = CalculateCovisitationMatrixBuy2Buy(n_splits, n_splits_concat, full_aid_list)
df_matrix_b2b_full = covisitation_buy2buy_full.generate_covisitation_matrix(click_data_full)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [14]:
# Check dataframe size and export to file.
size = df_matrix_b2b_full.memory_usage(deep='True').sum()
print(naturalsize(size))
df_matrix_b2b_full.to_parquet('matrix_buy2buy_test.parquet')

1.5 GB
